In [1]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 43.4 MB/s eta 0:00:00


In [2]:
%%writefile app.py
# Paste the full corrected code above here
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector

# Streamlit page title
st.title("Retail Order Analysis")

# Sidebar with selectbox
option = st.sidebar.selectbox("**Select Insight:**", ["Choose an Option","Top 10 Products by Revenue","Top 5 Cities by Profit Margin","Total Discount Given by Category","Average Sale Price by Category","Region with Highest Average Sale Price","Total Profit by Category","Top 3 Segments by Quantity",
"Average Discount by Region","Top Category by Profit","Total Revenue by Year","Top 5 Products by Orders","Top 5 Cities by Orders","Region with Lowest Revenue","Total Quantity Sold by Year","Top Category by Avg Discount","Top Segment by Revenue","Profit Margin by region","Average quantity sold per category","Category & Profit with Years","Months with sales"])

# Only run the SQL query when 'ins1' is selected
if option == "Average quantity sold per category":
    # Connect to MySQL database
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )

    mycursor = connection.cursor()
    mycursor.execute("""
        SELECT
            t3.category,
            ROUND(AVG(t4.quantity), 2) AS average_quantity_sold
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.category
        ORDER BY average_quantity_sold DESC;
    """)

    results = mycursor.fetchall()
    connection.close()

    # Convert results to DataFrame
    gf = pd.DataFrame(results, columns=["category", "average_quantity_sold"])
    gf["average_quantity_sold"] = pd.to_numeric(gf["average_quantity_sold"], errors='coerce')

    # Show data table
    st.subheader("Data Table: Average Quantity Sold per Category")
    st.dataframe(gf)

    # Show bar chart
    st.subheader("Bar Chart: Average Quantity Sold")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(gf["category"], gf["average_quantity_sold"], color="skyblue")
    ax.set_title("Average Quantity Sold per Product Category")
    ax.set_xlabel("Category")
    ax.set_ylabel("Average Quantity Sold")
    plt.xticks(rotation=45)
    plt.tight_layout()
    st.pyplot(fig)

if option == "Category & Profit with Years":
    # Connect to MySQL database
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )

    mycursor = connection.cursor()
    query = '''
        SELECT
            t3.category,
            YEAR(t3.order_date) AS order_year,
            ROUND(SUM(t4.profit), 2) AS total_profit
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.category, order_year
        ORDER BY t3.category, order_year;
    '''
    mycursor.execute(query)
    results = mycursor.fetchall()
    connection.close()

    # Convert results to DataFrame
    gf = pd.DataFrame(results, columns=["category", "order_year", "total_profit"])
    gf["total_profit"] = pd.to_numeric(gf["total_profit"], errors='coerce')

    # Show data table
    st.subheader("Data Table: Category with Profit by Year")
    st.dataframe(gf)

    # Line chart using matplotlib (grouped by category)
    st.subheader("Line Chart: Yearly Profit by Category")
    fig, ax = plt.subplots(figsize=(10, 6))

    for category in gf["category"].unique():
        category_data = gf[gf["category"] == category]
        ax.plot(category_data["order_year"], category_data["total_profit"], marker='o', label=category)

    ax.set_title("Yearly Profit Trends by Category")
    ax.set_xlabel("Year")
    ax.set_ylabel("Total Profit")
    ax.legend(title="Category", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(rotation=45)
    plt.tight_layout()
    st.pyplot(fig)

if option == "Months with sales":
    # Connect to DB
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()
    mycursor.execute("""
        SELECT
            YEAR(t3.order_date) AS order_year,
            MONTH(t3.order_date) AS order_month,
            ROUND(SUM(t4.sale_price), 2) AS total_sales
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY order_year, order_month
        ORDER BY order_year, order_month;
    """)
    result = mycursor.fetchall()
    connection.close()

    # DataFrame setup
    gf = pd.DataFrame(result, columns=["order_year", "order_month", "total_sales"])
    gf["total_sales"] = pd.to_numeric(gf["total_sales"], errors='coerce')

    # Show as data table
    st.subheader("Monthly Sales Table by Year")
    st.dataframe(gf)

    # Plot chart using matplotlib
    st.subheader("Line Chart: Monthly Sales by Year")
    fig, ax = plt.subplots(figsize=(12, 6))

    # Plot each year as a separate line
    for year in sorted(gf["order_year"].unique()):
        data = gf[gf["order_year"] == year]
        ax.plot(data["order_month"], data["total_sales"], marker='o', label=str(year))

    ax.set_title("Monthly Sales Trend by Year")
    ax.set_xlabel("Month")
    ax.set_ylabel("Total Sales")
    ax.set_xticks(range(1, 13))
    ax.legend(title="Year")
    plt.tight_layout()
    st.pyplot(fig)

if option == "Profit Margin by region":
    # Connect to MySQL
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # SQL Query: Profit margin by region
    query = """
        SELECT
            t3.region,
            ROUND(SUM(t4.profit) / SUM(t4.sale_price) * 100, 2) AS profit_margin_percent
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.region
        HAVING SUM(t4.sale_price) > 0
        ORDER BY profit_margin_percent DESC;
    """
    mycursor.execute(query)
    results = mycursor.fetchall()
    connection.close()

    # Convert results to DataFrame
    df = pd.DataFrame(results, columns=["region", "profit_margin_percent"])
    df["profit_margin_percent"] = pd.to_numeric(df["profit_margin_percent"], errors="coerce")

    # Show Data Table
    st.subheader("Data Table: Profit Margin by Region")
    st.dataframe(df)

    # Show Bar Chart
    st.subheader("Bar Chart: Profit Margin (%) by Region")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df["region"], df["profit_margin_percent"], color="mediumseagreen")
    ax.set_title("Profit Margin Percentage by Region")
    ax.set_xlabel("Region")
    ax.set_ylabel("Profit Margin (%)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    st.pyplot(fig)

if option == "Top Segment by Revenue":
    # Connect to MySQL
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # Execute the query
    mycursor.execute("""
        SELECT
            t3.segment,
            ROUND(SUM(t4.sale_price), 2) AS total_revenue
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.segment
        ORDER BY total_revenue DESC
        LIMIT 1;
    """)
    result = mycursor.fetchall()
    connection.close()

    # Convert to DataFrame
    df = pd.DataFrame(result, columns=["segment", "total_revenue"])

    # Display result as table
    st.subheader("Top Segment by Total Revenue")
    st.dataframe(df)

if option == "Top Category by Avg Discount":
    # Connect to MySQL
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # Execute the query
    mycursor.execute("""
        SELECT
            t3.category,
            ROUND(AVG(t4.discount_percent), 2) AS average_discount_percent
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.category
        ORDER BY average_discount_percent DESC
        LIMIT 1;
    """)
    result = mycursor.fetchall()
    connection.close()

    # Convert to DataFrame
    df = pd.DataFrame(result, columns=["category", "average_discount_percent"])

    # Display result as table
    st.subheader("Top Category by Average Discount (%)")
    st.dataframe(df)

if option == "Total Quantity Sold by Year":
    # Connect to MySQL
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # SQL query
    mycursor.execute("""
        SELECT
            YEAR(t3.order_date) AS order_year,
            SUM(t4.quantity) AS total_quantity_sold
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY order_year
        ORDER BY order_year;
    """)
    results = mycursor.fetchall()
    connection.close()

    # Convert to DataFrame
    df = pd.DataFrame(results, columns=["order_year", "total_quantity_sold"])
    df["total_quantity_sold"] = pd.to_numeric(df["total_quantity_sold"], errors='coerce')

    # Display table
    st.subheader("Data Table: Total Quantity Sold by Year")
    st.dataframe(df)

    # Display bar chart
    st.subheader("Bar Chart: Total Quantity Sold Each Year")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df["order_year"].astype(str), df["total_quantity_sold"], color="salmon")
    ax.set_title("Total Quantity Sold by Year")
    ax.set_xlabel("Year")
    ax.set_ylabel("Quantity Sold")
    plt.xticks(rotation=45)
    plt.tight_layout()
    st.pyplot(fig)

if option == "Region with Lowest Revenue":
    # Connect to MySQL
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # Execute the query
    mycursor.execute("""
        SELECT
            t3.region,
            ROUND(SUM(t4.sale_price), 2) AS total_revenue
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.region
        ORDER BY total_revenue ASC
        LIMIT 1;
    """)
    result = mycursor.fetchall()
    connection.close()

    # Convert to DataFrame
    df = pd.DataFrame(result, columns=["region", "total_revenue"])
    df["total_revenue"] = pd.to_numeric(df["total_revenue"], errors="coerce")

    # Display table
    st.subheader("Data Table: Region with Lowest Total Revenue")
    st.dataframe(df)

if option == "Top 5 Cities by Orders":
    # Connect to DB
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # Execute query
    mycursor.execute("""
        SELECT
            t3.city,
            COUNT(*) AS total_orders
        FROM table3 t3
        GROUP BY t3.city
        ORDER BY total_orders DESC
        LIMIT 5;
    """)
    results = mycursor.fetchall()
    connection.close()

    # Convert to DataFrame
    df = pd.DataFrame(results, columns=["city", "total_orders"])
    df["total_orders"] = pd.to_numeric(df["total_orders"], errors="coerce")

    # Display Data Table
    st.subheader("Top 5 Cities by Total Orders")
    st.dataframe(df)

    # Pie Chart
    st.subheader("Pie Chart: Distribution of Orders Among Top 5 Cities")
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.pie(df["total_orders"], labels=df["city"], autopct='%1.1f%%', startangle=140)
    ax.axis("equal")
    st.pyplot(fig)

if option == "Top 5 Products by Orders":
    # Connect to MySQL
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # Execute SQL query
    mycursor.execute("""
        SELECT
            t4.product_id,
            COUNT(*) AS order_count
        FROM table4 t4
        GROUP BY t4.product_id
        ORDER BY order_count DESC
        LIMIT 5;
    """)
    results = mycursor.fetchall()
    connection.close()

    # Convert results to DataFrame
    df = pd.DataFrame(results, columns=["product_id", "order_count"])
    df["order_count"] = pd.to_numeric(df["order_count"], errors="coerce")

    # Display table
    st.subheader("Top 5 Products by Number of Orders")
    st.dataframe(df)

    # Simulated 3D pie chart using shadow
    st.subheader("Pie Chart: Top 5 Products by Orders")
    fig, ax = plt.subplots(figsize=(8, 6))
    wedges, texts, autotexts = ax.pie(
        df["order_count"],
        labels=df["product_id"],
        autopct="%1.1f%%",
        shadow=True,  # 3D-like shadow effect
        startangle=140
    )
    ax.axis("equal")  # Equal aspect ratio ensures that pie is drawn as a circle
    st.pyplot(fig)

if option == "Total Revenue by Year":
    # Connect to MySQL
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # Execute SQL query
    query = """
        SELECT
            YEAR(t3.order_date) AS order_year,
            ROUND(SUM(t4.sale_price), 2) AS total_revenue
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY order_year
        ORDER BY order_year;
    """
    mycursor.execute(query)
    results = mycursor.fetchall()
    connection.close()

    # Convert to DataFrame
    df = pd.DataFrame(results, columns=["order_year", "total_revenue"])
    df["total_revenue"] = pd.to_numeric(df["total_revenue"], errors="coerce")

    # Display table
    st.subheader("Total Revenue by Year")
    st.dataframe(df)

    # Display histogram chart
    st.subheader("Histogram: Revenue Distribution by Year")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df["order_year"], df["total_revenue"], color="orange", edgecolor="black")
    ax.set_title("Total Revenue per Year")
    ax.set_xlabel("Year")
    ax.set_ylabel("Revenue")
    plt.xticks(rotation=45)
    plt.tight_layout()
    st.pyplot(fig)

if option == "Top Category by Profit":
    # Connect to MySQL
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # Execute query to get top category by profit
    query = """
        SELECT
            t3.category,
            ROUND(SUM(t4.profit), 2) AS total_profit
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.category
        ORDER BY total_profit DESC
        LIMIT 1;
    """
    mycursor.execute(query)
    results = mycursor.fetchall()
    connection.close()

    # Convert to DataFrame
    df = pd.DataFrame(results, columns=["category", "total_profit"])

    # Display as table in Streamlit
    st.subheader("Top Category by Total Profit")
    st.dataframe(df)

if option == "Average Discount by Region":
    # Connect to MySQL
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # Execute SQL query
    query = """
        SELECT
            t3.region,
            ROUND(AVG(t4.discount_percent), 2) AS average_discount_percent
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.region
        ORDER BY average_discount_percent DESC;
    """
    mycursor.execute(query)
    results = mycursor.fetchall()
    connection.close()

    # Convert to DataFrame
    df = pd.DataFrame(results, columns=["region", "average_discount_percent"])
    df["average_discount_percent"] = pd.to_numeric(df["average_discount_percent"], errors="coerce")

    # Display as data table
    st.subheader("Average Discount Percentage by Region")
    st.dataframe(df)

    # Display as bar chart
    st.subheader("Bar Chart: Avg Discount (%) by Region")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df["region"], df["average_discount_percent"], color="orchid")
    ax.set_title("Average Discount Percentage by Region")
    ax.set_xlabel("Region")
    ax.set_ylabel("Avg Discount (%)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    st.pyplot(fig)

if option == "Top 3 Segments by Quantity":
    # Connect to MySQL
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # Execute the SQL query
    query = """
        SELECT
            t3.segment,
            SUM(t4.quantity) AS total_quantity
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.segment
        ORDER BY total_quantity DESC
        LIMIT 3;
    """
    mycursor.execute(query)
    results = mycursor.fetchall()
    connection.close()

    # Convert results to DataFrame
    df = pd.DataFrame(results, columns=["segment", "total_quantity"])
    df["total_quantity"] = pd.to_numeric(df["total_quantity"], errors="coerce")

    # Display the table
    st.subheader("Top 3 Segments by Quantity Sold")
    st.dataframe(df)

    # Plot a pie chart
    st.subheader("Pie Chart: Top 3 Segments by Quantity")
    fig, ax = plt.subplots()
    ax.pie(
        df["total_quantity"],
        labels=df["segment"],
        autopct='%1.1f%%',
        startangle=140,
        colors=plt.cm.Set2.colors
    )
    ax.axis('equal')
    plt.tight_layout()
    st.pyplot(fig)

if option == "Total Profit by Category":
    # Connect to MySQL database
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # SQL query: Total profit by category
    query = """
        SELECT
            t3.category,
            ROUND(SUM(t4.profit), 2) AS total_profit
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.category
        ORDER BY total_profit DESC;
    """
    mycursor.execute(query)
    results = mycursor.fetchall()
    connection.close()

    # Convert results to DataFrame
    df = pd.DataFrame(results, columns=["category", "total_profit"])
    df["total_profit"] = pd.to_numeric(df["total_profit"], errors="coerce")

    # Display the table
    st.subheader("Data Table: Total Profit by Category")
    st.dataframe(df)

    # Plot bar chart
    st.subheader("Bar Chart: Total Profit by Category")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df["category"], df["total_profit"], color="teal")
    ax.set_title("Total Profit by Product Category")
    ax.set_xlabel("Category")
    ax.set_ylabel("Total Profit")
    plt.xticks(rotation=45)
    plt.tight_layout()
    st.pyplot(fig)

if option == "Region with Highest Average Sale Price":
    # Connect to MySQL database
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # SQL query
    query = """
        SELECT
            t3.region,
            ROUND(AVG(t4.sale_price), 2) AS average_sale_price
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.region
        ORDER BY average_sale_price DESC
        LIMIT 1;
    """
    mycursor.execute(query)
    result = mycursor.fetchall()
    connection.close()

    # Convert result to DataFrame
    df = pd.DataFrame(result, columns=["region", "average_sale_price"])
    df["average_sale_price"] = pd.to_numeric(df["average_sale_price"], errors="coerce")

    # Show Data Table
    st.subheader("Data Table: Region with Highest Average Sale Price")
    st.dataframe(df)

if option == "Average Sale Price by Category":
    # Connect to MySQL database
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # SQL Query
    query = """
        SELECT
            t3.category,
            ROUND(AVG(t4.sale_price), 2) AS average_sale_price
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.category
        ORDER BY average_sale_price DESC;
    """
    mycursor.execute(query)
    results = mycursor.fetchall()
    connection.close()

    # Create DataFrame
    df = pd.DataFrame(results, columns=["category", "average_sale_price"])
    df["average_sale_price"] = pd.to_numeric(df["average_sale_price"], errors="coerce")

    # Display table
    st.subheader("Data Table: Average Sale Price by Category")
    st.dataframe(df)

    # 3D-style bar chart using matplotlib
    from mpl_toolkits.mplot3d import Axes3D
    import numpy as np

    st.subheader("3D Bar Chart: Average Sale Price by Category")
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    xpos = np.arange(len(df))
    ypos = np.zeros(len(df))  # y-axis remains flat (2D)
    zpos = np.zeros(len(df))
    dx = np.ones(len(df)) * 0.5
    dy = np.ones(len(df)) * 0.5
    dz = df["average_sale_price"].values

    ax.bar3d(xpos, ypos, zpos, dx, dy, dz, color="steelblue", shade=True)
    ax.set_xticks(xpos)
    ax.set_xticklabels(df["category"], rotation=45, ha="right")
    ax.set_ylabel("Category")
    ax.set_zlabel("Average Sale Price")
    ax.set_title("Average Sale Price by Category (3D View)")

    st.pyplot(fig)

if option == "Total Discount Given by Category":
    # Connect to MySQL database
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # SQL query to calculate total discount by category
    query = """
        SELECT
            t3.category,
            ROUND(SUM((t4.discount_percent / 100) * t4.list_price * t4.quantity), 2) AS total_discount_given
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.category
        ORDER BY total_discount_given DESC;
    """
    mycursor.execute(query)
    results = mycursor.fetchall()
    connection.close()

    # Convert to DataFrame
    df = pd.DataFrame(results, columns=["category", "total_discount_given"])
    df["total_discount_given"] = pd.to_numeric(df["total_discount_given"], errors="coerce")

    # Show table
    st.subheader("Data Table: Total Discount Given by Category")
    st.dataframe(df)

    # Bar chart
    st.subheader("Bar Chart: Total Discount Given")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df["category"], df["total_discount_given"], color="orange")
    ax.set_title("Total Discount Given by Category")
    ax.set_xlabel("Category")
    ax.set_ylabel("Total Discount Given (in $)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    st.pyplot(fig)

if option == "Top 5 Cities by Profit Margin":
    # Connect to MySQL database
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    # SQL query for top 5 cities by profit margin
    query = """
        SELECT
            t3.city,
            ROUND(SUM(t4.profit) / SUM(t4.sale_price) * 100, 2) AS profit_margin_percent
        FROM table4 t4
        JOIN table3 t3 ON t4.order_id = t3.order_id
        GROUP BY t3.city
        HAVING SUM(t4.sale_price) > 0
        ORDER BY profit_margin_percent DESC
        LIMIT 5;
    """
    mycursor.execute(query)
    results = mycursor.fetchall()
    connection.close()

    # Convert to DataFrame
    df = pd.DataFrame(results, columns=["city", "profit_margin_percent"])
    df["profit_margin_percent"] = pd.to_numeric(df["profit_margin_percent"], errors="coerce")

    # Display data table
    st.subheader("Data Table: Top 5 Cities by Profit Margin (%)")
    st.dataframe(df)

    # 3D Pie Chart (simulated with shadow effect)
    st.subheader("3D Pie Chart: Profit Margin by Top Cities")
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.pie(df["profit_margin_percent"],
           labels=df["city"],
           autopct="%1.1f%%",
           shadow=True,
           startangle=140)
    ax.set_title("Top 5 Cities by Profit Margin (%)", fontsize=14)
    st.pyplot(fig)

if option == "Top 10 Products by Revenue":
    # Connect to MySQL
    connection = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2ZsEqVVabLFWFUp.root",
        password="16l1x13dLBPp8jug",
        database="retail_order",
    )
    mycursor = connection.cursor()

    query = """
        SELECT
            product_id,
            SUM(sale_price) AS total_revenue
        FROM table4
        GROUP BY product_id
        ORDER BY total_revenue DESC
        LIMIT 10;
    """
    mycursor.execute(query)
    out = mycursor.fetchall()
    connection.close()

    # Convert to DataFrame
    df = pd.DataFrame(out, columns=["product_id", "total_revenue"])
    df["total_revenue"] = pd.to_numeric(df["total_revenue"], errors='coerce')

    # Show table
    st.subheader("Data Table: Top 10 Products by Revenue")
    st.dataframe(df)

    # Normal bar chart using matplotlib
    st.subheader("Bar Chart: Top 10 Products by Revenue")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df["product_id"], df["total_revenue"], color='cornflowerblue')
    ax.set_title("Top 10 Products by Revenue")
    ax.set_xlabel("Product ID")
    ax.set_ylabel("Total Revenue")
    plt.xticks(rotation=45)
    plt.tight_layout()
    st.pyplot(fig)







Writing app.py


In [3]:
# Install necessary packages
!pip install streamlit pyngrok --quiet

# Set ngrok auth token (Replace YOUR_NGROK_AUTH_TOKEN with your actual token)
!ngrok authtoken 2wAoCXQJI2pt4JigixFLLwYUbu1_2LmrCqL81EajP8hCu2DPN


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
import os
import time
from pyngrok import ngrok

# Expose the port using ngrok
public_url = ngrok.connect(8501)
print(f"Streamlit is running at: {public_url}")



# Start Streamlit in the background
!streamlit run app.py --server.port 8501 --server.headless true &

# Wait for Streamlit to start
time.sleep(1)


Streamlit is running at: NgrokTunnel: "https://f78c-34-23-197-123.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.23.197.123:8501

  Stopping...
